In [13]:
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv
load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN") # reads .env file with ACCESS_TOKEN=<your hugging face access token>

model_id = "Datascience-Lab/GPT2-small"
tokenizer = AutoTokenizer.from_pretrained(model_id, token=ACCESS_TOKEN)
# quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4")

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             token=ACCESS_TOKEN,
                                             trust_remote_code=True
                                             )
model.eval()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"using {device}")

c:\Users\mazpr\anaconda3\envs\openai\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mazpr\.cache\huggingface\hub\models--google--canine-c. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


using cpu


In [20]:
from openai import OpenAI
import os

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
def get_completion(message):
    chat_completion = client.chat.completions.create(
        messages=message,
        model="gpt-3.5-turbo",
    )

    return chat_completion.choices[0].message.content

In [19]:
def inference(question: str, context: str):

    if context == None or context == "":
        prompt = f"""Give a detailed answer to the following question. Question: {question}"""
    else:
        prompt = f"""Using the information contained in the context, give a detailed answer to the question.
            Context: {context}.
            Question: {question}"""
    chat = [
        {"role": "user", "content": prompt},
        # { "role": "model", "content": "Recurrent Attention (RAG)** is a novel neural network architecture specifically designed" }
    ]

    messages = [
    {"role": "system", "content": "You are a helpful assisstent that knows everything about Mazi Prima ."},
    {"role": "user", "content": prompt},]
    return get_completion(messages)

In [8]:
inference("Hello!", "")

'I am a helpful virtual assistant here to assist you with any questions or tasks you may have. How can I help you today?'

# Document Loading and Splitting

In [1]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("dataset/about_me.pdf"),
]
pages = []
for loader in loaders:
    pages.extend(loader.load())

In [3]:
from langchain.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=12)
docs = text_splitter.split_documents(pages)

In [13]:
import numpy as np
np.save("docs.npy", docs)

In [17]:
docs = np.load("docs.npy", allow_pickle=True)

# Embeddings and Vector Store

In [6]:
import numpy as np
from langchain_community.embeddings import (
    HuggingFaceEmbeddings
)
from langchain.vectorstores import FAISS


encoder = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2', model_kwargs = {'device': "cpu"})
faiss_db = FAISS.from_documents(docs, encoder)


c:\Users\mazpr\anaconda3\envs\openai\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\mazpr\anaconda3\envs\openai\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
embeddings = encoder.embed_query("How are you?")

In [14]:
def get_context(question):
    retrieved_docs = faiss_db.similarity_search(question, k=5)
    context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
    return context

In [32]:
question = "What's her bachelors degree?"
retrieved_docs = faiss_db.similarity_search(question, k=2)
context = "".join(doc.page_content + "\n" for doc in retrieved_docs)
print(context)

 complex topics accessible to young students. In smaller groups or 
one-on-one settings, students were more comfortable asking questions, significantly 
enhancing their learning experience.  
Education:  
Mazi holds a Bachelor of Science degree in Mathematics from Institut Teknologi Bandung, 
where he completed his und ergraduate thesis on "ASABRI and JIWASRAYA Stock Portfolio 
Optimization using Adaptive Spiral Optimization Method," achieving a GPA of 3.16 out of 
4.00.  
Technical Skills:  
His technical
.  
Technical Skills:  
His technical skills encompass a wide range of tools and technologies, including Pyth on (with 
libraries such as Dash Plotly, Streamlit, Pandas, Sklearn, Keras, SHAP, Pytest, Flask, and 
more), PySpark, SQL, Open AI, GCP, Azure Machine Learning, Azure Open AI, Tableau, 
and Power BI. His expertise in data analysis includes statistical analyti cs, data preparation, 
data visualization, and dashboard building. In data modeling, Mazi excels in developing 
recomm

In [33]:
question = "Whats her bachelors degree?"
context = get_context(question)
print(inference(question=question, context=context))

Mazi holds a Bachelor of Science degree in Mathematics from Institut Teknologi Bandung.


In [23]:
context

"  \nMazi aims to become a T-shaped Data Scientist, with deep expertise in NLP, an area he is \nparticularly passionate about and wishes to explore further.  \nPersonal Interests:  \nBeyond his technical capabilities, Mazi is a proactive and innovative professional who \nconsistently seeks to leverage data to drive business success and improve operational \nefficiencies. His interests include reading and hiking, which provide him with a well -rounded \nperspective and a balanced approach to both his professional and personal life.  \nFor more inform ation about Mazi Prima Re\nMazi Prima Reza - Data Scientist  \nMazi Prima Reza is a skilled Data Scientist with over three years of experience in the data \nfield, currently working at Metrodata in Jakarta, Indonesia. His professional journey is \nmarked by a strong background in handling diverse and end -to-end projects, ranging from \ndata analysis to the implementation of advanced generative AI technologies. Maz i is \ndedicated to build